In [3]:
import pandas as pd;
import numpy as np;
import serial, time 
import json

In [4]:
#!pip install --upgrade pandas

In [2]:
# connect to Track
ser = serial.Serial('/dev/ttyUSB0', 9600, timeout = None)
print(ser.name)

/dev/ttyUSB0


In [4]:
# csv or excel
# name, car number, den, category

# "Race number" car number" x4, "lane" for heats

In [7]:
# Read from CSV to load Races
races = pd.read_csv('races2021.csv')
races = races.set_index('Race Number')
#races.head()

In [8]:
#races.loc[2,'Lane 2'] = 1
races.head(15)

,Lane 1,Lane 2,Lane 3,Lane 4
Race Number,,,,
1,1,2,3,4
2,5,6,7,8
3,11,12,13,14
4,15,16,17,18
5,2,5,10,1
6,3,4,6,9
7,19,13,11,12
8,16,20,21,17
9,4,1,5,6


In [ ]:
# Read list of racers and cars from CSV

racers = pd.read_csv('Racers.csv')
racers = racers.set_index('Number')
racers.head(25)

In [8]:
# ### only for WEB
#create Race Output for Upcoming race
# for race, build details for lane
# get lane 1 car
race = {}
lane = 1
for carNum in races.loc[1]:
    name = racers.loc[carNum,'Name']
    #print(lane, carNum, name)
    details = {'car': str(carNum), 'name': name}
    trackName = 'lane' + str(lane)
    race[trackName] = details
    lane = lane +1

nextup = {'nextUp': race}
with open('src/nextup.json', 'w') as outfile:
    json.dump(nextup, outfile)


In [17]:
# Set Race
current_race = 1

In [18]:
# resultsdf = pd.DataFrame(columns=["Race","Lane","Car","Time","Scout","Den"])
#resultsdf = pd.DataFrame(columns=["Race","Lane","Car","Time"])

resultsdf = pd.DataFrame(columns=['race', 'lane', 'car', 'name', 'time', 'place', 'den', 'category'])
resultsdf.head()

,race,lane,car,name,time,place,den,category


# Start of the Race process

In [114]:
# set current race if needed
current_race = 13

In [115]:
print(current_race)

13


In [23]:
# show curret race
current_contestants = races.loc[current_race]
print(races.loc[current_race])

Lane 1    1
Lane 2    2
Lane 3    3
Lane 4    4
Name: 1, dtype: int64


In [10]:
new1 = current_contestants[current_contestants == 0]
print(new1)

Series([], Name: 1, dtype: int64)


In [ ]:
# Reverse Lane order
command = 'ov1'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
results = ser.readline()
print(laneresults)


# confirm Lane Results

In [11]:
# Reset Track if needed
#Reset Track
command = 'r'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)

3

## Run Race


### Call out # of lanes and race number


In [158]:
print('In Race ' + str(current_race) + ' the following cars are racing: \n'+ str(races.loc[current_race]))

In Race 21 the following cars are racing: 
Lane 1    11
Lane 2    13
Lane 3    15
Lane 4    17
Name: 21, dtype: int64


In [30]:
# Turn on all Lanes   om# is used to turn off track
command = 'om0'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
returnmessage = ser.readline()
print(returnmessage.decode())

1=9.9999  2=9.9999  3=9.9999  4=9.9999  



In [13]:
# Turn off tracks that are not being used
# lanesToBlock = [1,4]
#lanesToBlock = [3]

for x in (lanesToBlock):
    #lanenum = (x[4:5])
    #command = 'om' + lanenum +'\r\n'
    command = 'om' + str(x) +'\r\n'
    print(command)
    newcommand = command.encode('ascii')
    ser.write(newcommand)
    returnmessage = ser.readline()
    print(returnmessage.decode())

om3

?



In [ ]:
#current_race = 4

In [86]:

# Get Race Results
# get race results
command = 'rg'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
results = ser.readline()
laneresults = results.decode().split()
print(laneresults)


# confirm Lane Results

['1=3.27452', '2=3.24041', '3=3.35533', '4=3.39354']


# Force finish race 

In [159]:
# force race end
command = 'ra'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
results = ser.readline()



In [160]:

# Get Race Results
# get race results
command = 'rp'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
results = ser.readline()
laneresults = results.decode().split()
print(laneresults)


['1=7.16754', '2=4.47692', '3=4.78003', '4=3.40431']


In [161]:
#laneresults = ['1=1.17041', '2=5.41592', '3=3.54623', '4=3.72464']
#laneresults = ['1=3.06202', '2=3.15254', '3=3.03381', '4=3.12463']
print('Confirm Results: \n' + str(laneresults))

Confirm Results: 
['1=7.16754', '2=4.47692', '3=4.78003', '4=3.40431']


## Write Results to results df


## Calculate leaders and publish leaders based on average time

In [162]:
race = {}
for laneresult in laneresults:
    lane = laneresult[0]
    car = races.loc[int(current_race),'Lane '+lane]
    time = float(laneresult[2:8])
    name = racers.loc[car,'Name']
    den = racers.loc[car,'Den']
    category = racers.loc[car,'Category']
    place = laneresult[8:9]
    
    # Create results json
    details = {'race' : current_race , 'lane' : int(lane), 'car': str(car), 'name': name, 'time': time, 'place': place, 'den': den, 'category': category} 
    trackName = 'lane' + str(lane)
    race[trackName] = details
    
 
    # add den, category
    resultsdf = resultsdf.append(details, ignore_index=True)
    #print(details)
   
current_race = current_race +1
resultsdf.to_csv('resultsFile2021.csv')


In [ ]:
## End of Race

# Final Results

In [ ]:
## Den Results
den_results = backup.groupby(['den','car','name']) \
       .agg({'car':'size', 'time':'mean'}) \
       .rename(columns={'car':'Races Completed'}) \
       .sort_values('time', ascending=True)
       #.reset_index()
print(den_results)

In [15]:
den_results.to_csv('FinalScores.csv')

In [ ]:
## Den Results
den_results = backup.groupby(['lane','car','name']) \
       .agg({'car':'size', 'time':'mean'}) \
       .rename(columns={'car':'Races Completed'}) \
       .sort_values('time', ascending=True)
       #.reset_index()
print(den_results)

In [34]:
# Get top cars
standings = resultsdf.groupby('car') \
    .apply(lambda x: x.drop([x['time'].idxmax()]))\
    .rename_axis(['time','time'])\
    .groupby('car')\
    .agg({'car':'size', 'time':'mean'}) \
    .rename(columns={'car':'Races Completed','time':'Average Time'}) \
    .sort_values('Average Time', ascending=True)


#standings = standings.sort_values('Average Time', ascending=True)
print(standings.iloc[0:4])

Empty DataFrame
Columns: [Races Completed, Average Time]
Index: []


# Extra commands

In [35]:
# Turn on Set Place   indicatorto #
command = 'op2'+'\r\n'
newcommand = command.encode('ascii')
ser.write(newcommand)
returnmessage = ser.readline()
print(returnmessage.decode())

NameError: name 'ser' is not defined

In [ ]:
backup = pd.read_csv('resultsFile2021.csv')
backup.head()

In [39]:
leaderdf.head()

NameError: name 'leaderdf' is not defined

In [26]:
## extra functions update or rewrite Json for current race if needed to export
currentRace = {'currentRace': race}
with open('src/current.json', 'w') as outfile:
    json.dump(currentRace, outfile)


## update or rewrite Json for current race.  
avg = resultsdf.groupby(['car','name']) \
       .agg({'car':'size', 'time':'mean'}) \
       .rename(columns={'car':'Races Completed'}) \
       .reset_index()

avgtimes = avg.to_dict(orient='records')
avgTimes = {'averageTimes': avgtimes}
with open('src/avg.json', 'w') as outfile:
    json.dump(avgTimes, outfile)

top = resultsdf.groupby(['car','name']) \
       .agg({'time':'min'}) \
       .reset_index()

toptimes = top.to_dict(orient='records')

## update or rewrite Json for current race.  
topTimes = {'topSpeeds': toptimes}
with open('src/top.json', 'w') as outfile:
    json.dump(topTimes, outfile)
    
# create Race Output for Upcoming race
# for race, build details for lane
# get lane 1 car
race = {}
lane = 1
for carNum in races.loc[current_race]:
    name = racers.loc[carNum,'Name']
    #print(lane, carNum, name)
    details = {'car': str(carNum), 'name': name}
    trackName = 'lane' + str(lane)
    race[trackName] = details
    lane = lane +1

nextup = {'nextUp': race}
with open('src/nextup.json', 'w') as outfile:
    json.dump(nextup, outfile)


In [ ]:
# Find Car by race and lane
car = racedf.loc[1,'lane1']
print (car)

In [ ]:
results = ser.readline()

In [ ]:
times = avg.loc[:,['car','name','Average Time']]
times = times.reset_index()
times.head()

In [ ]:
# delete race result or duplicate race result

In [ ]:
# write out rsults df and load from csv

In [ ]:
def serialwrite(code):
    command = code +'\r\n'
    #newcommand = command.encode('ascii')
    #ser.write(newcommand)
    #results = ser.readline()
    #laneresults = results.decode().split()
    #print(laneresults)
    return results

serialwrite('rg')

In [ ]:
currentrace['currentRace']['lane1']['time'] ='3.012'
lastRace = currentrace['currentRace']
type(lastRace)